In [1]:
%autosave 60

Autosaving every 60 seconds


In [2]:
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import decomposition
from sklearn.metrics import accuracy_score, roc_auc_score,recall_score,make_scorer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler  
from sklearn.neural_network import MLPClassifier


# стороняя библиотека для работы с несбалансированными датасетами
# pip install imblearn
from imblearn.over_sampling import SMOTE, ADASYN

# настройки отображения графиков
# %config InlineBackend.figure_format = 'svg' 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="ticks", color_codes=True)
%matplotlib inline

# увеличим  размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 8,7

from tqdm import tqdm_notebook

# для воспроизводимости
r_state = 11

In [3]:
data_path = './data/creditcard.csv'

In [ ]:
df = pd.read_csv(data_path)

Проверим, что все считалось должным образом:

In [ ]:
df.head()

Сперва посмотрим на распределение меток у целевого класса:

In [ ]:
# sns.countplot(df['Class']);

Оценим, насколько мошеннические транзакции коррелируют со временем:

In [ ]:
# sns.distplot(df[df['Class']==1]['Time'],100);

Как видим, распределение числа мошеннических транзакций во времени имеет примерно равномерный характер, следовательно, можно исключить из выборки этот признак. Тем более, что это время отсчитывалось, начиная от первой транзакции, следовательно, оно не имеет ценности

Посмотрим как пространство признаков изображается при двумерной проекции(на примере признаков 'V11', 'V12', 'V13', 'V14', 'V15') :

In [ ]:
# %%time

# rcParams['figure.figsize'] = 12,12
# sns_plot = sns.pairplot(df,vars=['V11', 'V12', 'V13', 'V14', 'V15'],hue="Class",markers=["o", "s"])
# rcParams['figure.figsize'] = 8,7

Для получения качественных оценок работы алгоритма, я буду сравнивать такие метрики как:
- **Recall score**

   Это оценка вида: 
   
   $$score = {tp \over tp + fn}$$
   
   tp - число предсказаний, когда модель верно обнаружила нужные транзакции.
   
   fn - число предсказаний, когда модель не распознала поддельный перевод.
   
   Чем ближе это значение к 1, тем меньше модель пропускает поддельных транзакций. Но, возможно, она будет помечать некоторые переводы как поддельные, хотя они таковыми не являются. Следующие метрики должны помочь проконтролировать этот аспект.
   
   
- **Accuracy**

   Доля верных ответов, будем его тоже учитывать.
   
   
- **ROC AUC score**

   Площадь, ограниченная ROC-кривой и осью доли ложных положительных классификаций.
   Эта метрика позволяет судить о качестве бинарной классификации и в совокупности с Recall score должна дать нам наглядное представление о работе классификатора.

In [ ]:
df = df.sample(n=10000,random_state = r_state)

С помощью этой функции будем выводить информацию обо всех метриках для данной выборки и классификатора:

In [ ]:
def all_metrics_validation(clf,X,Y_true):
    print("Accuracy score: "+ str(accuracy_score(Y_true,clf.predict(X))))
    print("ROC AUC score: "+ str(roc_auc_score(Y_true,clf.predict_proba(X)[:,1]))) # нас интересуют положительные результаты
    print("Recall score: "+ str(recall_score(Y_true,clf.predict(X))))

Для того, что получить сбалансированный датасет, я буду использовать алгоритм ADASYN(Adaptive Synthetic Sampling),поскольку SMOTE (Synthetic Minority Over-sampling Technique) создает больше примеров внутри кластера, а ADASYN создает больше синтетических примеров на границе. 

В случае, когда данные генерируются ADASYN, модель становится более "подозрительной", поскольку будет лучше опознавать мошеннические транзакции на границе двух кластеров, ведь ADASYN может расширить границы имеющего кластера. Хотя это и может привести к ошибкам рода False Positive, мне кажется, что такой подход будет надежнее и более подходит для предотвращения мошенничества.

Производить оценку метрик я буду на трех датасетах.

1. На исходном
2. На тестовой части датасета, сгенерированного с помощью алгоритма ADASYN. 
3. На датасете, полученным контатенацией равных долей поддельных транзакций и неподдельных. То есть, я соединил все поддельные транзакции с тем же числом неподдельных (функция ниже)

In [ ]:
# Соединим строки поддельных с переводов исходного датасета с таким же количеством неподдельных
def create_balanced_data_from_initial(data):
    fraud = data[(data['Class']==1)]
    not_fraud = df.sample(data[(data['Class']==0)],n=len(fraud),random_state = r_state)
    new_df=pd.concat([fraud,not_fraud])      
    X = new_df.drop(['Class','Time'], axis=1)
    Y = new_df['Class']
    return X,Y

In [ ]:
# Применение алгоритма ADASYN
def create_balanced_data_adasyn(x,y):
    sm = ADASYN(random_state=12, ratio = 'minority')
    X,Y = sm.fit_sample(x, y)
    X = pd.DataFrame(X,columns = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount'] )
    Y = pd.DataFrame(Y,columns = ['Class'])
    return X,Y

In [ ]:
# Оценка модели на указанных датасетах
def estimate_model(clf, X_balanced, y_balanced, X_original,y_original, X_test, y_test):
    print('\nМетрики на оригинальном датасете: \n')
    all_metrics_validation(clf,X_original,y_original)
    print('\nМетрики на тестовой части :\n')
    all_metrics_validation(clf,X_test, y_test)
    print('\nМетрики на небольшой сбалансированой части из оригинального датасета:\n')
    all_metrics_validation(clf,X_balanced, y_balanced)
    

In [ ]:
# Разобьем данные на матрицу Х и вектор ответов Y
X_original = df.drop(['Class','Time'], axis=1)
y_original = df['Class']

In [ ]:
# Разделим на тестовую и тренировочную
X_train, X_test, y_train, y_test = train_test_split(X_original, y_original, test_size=0.3)

In [ ]:
# Сбалансированный датасет из исходного
X_balanced, y_balanced = create_balanced_data_from_initial(df)

Далее, при генерации данных ADASYN, необходимо сначала отделить тестовую и проверочную выборки, а затем для каждой сгенерировать новые данные. Тк в противном случае, если сначала создать новых данных, а потом разделить выборку, то синтетические данные попадут в проверочный сет.

In [ ]:

X_train_adasyn, y_train_adasyn = create_balanced_data_adasyn(X_train, y_train)

In [ ]:
# X_test_adasyn, y_test_adasyn = create_balanced_data_adasyn(X_test, y_test)

In [ ]:
sns.countplot(y_train_adasyn['Class']);y_train_adasyn['Class'].value_counts()

# Случайный лес

In [ ]:
%%time
parameters = {'max_features': ['auto'], 'min_samples_leaf': range(15,20,2),'max_depth': range(3,7,1),'n_estimators':range(10,50,10),'n_jobs':[-1]}

skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=r_state)

rfc = RandomForestClassifier()

gcv = GridSearchCV(rfc, parameters, n_jobs=-1, cv=skf, verbose=1,scoring='recall')

gcv.fit(X_train_adasyn, y_train_adasyn)

In [ ]:
clf = gcv.best_estimator_
gcv.best_params_ , gcv.best_score_

In [ ]:
estimate_model(clf, X_balanced, y_balanced, X_original,y_original, X_test, y_test)

# Метод ближайших соседей

In [ ]:
%%time
parameters = {'n_neighbors': range(10,30,5),'n_jobs':[-1]}

skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=r_state)

knn = KNeighborsClassifier()

gcv = GridSearchCV(knn, parameters, n_jobs=-1, cv=skf, verbose=1,scoring='recall')

gcv.fit(X_train_adasyn, y_train_adasyn)

In [ ]:
clf = gcv.best_estimator_
gcv.best_params_ , gcv.best_score_

In [ ]:
estimate_model(clf,X_balanced, y_balanced, X_original,y_original, X_test, y_test)

# Логистическая регрессия

In [ ]:
%%time
logit = LogisticRegression(n_jobs=-1)       

parameters = {'C': np.linspace(10,30,50),}

skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=r_state)

gcv = GridSearchCV(logit, parameters, n_jobs=-1, cv=skf, verbose=1,scoring='recall')

gcv.fit(X_train_adasyn, y_train_adasyn)

In [ ]:
clf = gcv.best_estimator_
gcv.best_params_ , gcv.best_score_

In [ ]:
estimate_model(clf, X_balanced, y_balanced, X_original,y_original, X_test, y_test)

Нейросеть

Поскольку перед нами не стоит задача распознавания изображений, у нас не так много данных и они не имею слишком большой размерности, то воспользуемся встроенной в SKlearn нейросетью - MultiLayerClassifier

In [ ]:
%%time

scaler = StandardScaler() 
scaler.fit_transform(X_train_adasyn)


mlp = MLPClassifier(random_state=r_state)
parameters = {'activation':['logistic'], 'alpha':[1e-5],'hidden_layer_sizes':[(50,),(60,),(200,)],'learning_rate':['adaptive']}

skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=r_state)

gcv = GridSearchCV(mlp, parameters, n_jobs=-1, cv=skf, verbose=1,scoring='recall')

gcv.fit(X_train_adasyn, y_train_adasyn)

In [ ]:
clf = gcv.best_estimator_
gcv.best_params_ , gcv.best_score_

In [ ]:
estimate_model(clf, scaler.transform(X_balanced),y_balanced, scaler.transform(X_original),y_original, scaler.transform(X_test), y_test)